If you are just a student and dont have enough computation power and/or training set  
Just use a pre-trained model as we do  
Using pre-trained model from there: https://github.com/balancap/SSD-Tensorflow  

In [1]:
import os
import math
import random

import numpy as np
import tensorflow as tf
import cv2

slim = tf.contrib.slim

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [3]:
import sys
this_dir = os.getcwd()
lib_path = os.path.join(this_dir, 'SSD-Tensorflow')
sys.path.append(lib_path)

from nets import ssd_vgg_300, ssd_common, np_methods
from preprocessing import ssd_vgg_preprocessing
import visualization

In [4]:
isess = tf.InteractiveSession()

In [5]:
# Input placeholder.
net_shape = (300, 300)
data_format = 'NHWC'
img_input = tf.placeholder(tf.uint8, shape=(None, None, 3))
# Evaluation pre-processing: resize to SSD net shape.
image_pre, labels_pre, bboxes_pre, bbox_img = ssd_vgg_preprocessing.preprocess_for_eval(
    img_input, None, None, net_shape, data_format, resize=ssd_vgg_preprocessing.Resize.WARP_RESIZE)
image_4d = tf.expand_dims(image_pre, 0)

# Define the SSD model.
reuse = True if 'ssd_net' in locals() else None
ssd_net = ssd_vgg_300.SSDNet()
with slim.arg_scope(ssd_net.arg_scope(data_format=data_format)):
    predictions, localisations, _, _ = ssd_net.net(image_4d, is_training=False, reuse=reuse)

# Restore SSD model.
ckpt_filename = this_dir + '/data/SSD-VGG/VGG_VOC0712_SSD_300x300_ft_iter_120000.ckpt'
isess.run(tf.global_variables_initializer())
saver = tf.train.Saver()
saver.restore(isess, ckpt_filename)

# SSD default anchor boxes.
ssd_anchors = ssd_net.anchors(net_shape)

INFO:tensorflow:Restoring parameters from /home/jovyan/work/NetsLab/data/SSD-VGG/VGG_VOC0712_SSD_300x300_ft_iter_120000.ckpt


In [6]:
# Main image processing routine.
def process_image(img, select_threshold=0.5, nms_threshold=.45, net_shape=(300, 300)):
    # Run SSD network.
    rimg, rpredictions, rlocalisations, rbbox_img = isess.run([image_4d, predictions, localisations, bbox_img],
                                                              feed_dict={img_input: img})
    
    # Get classes and bboxes from the net outputs.
    rclasses, rscores, rbboxes = np_methods.ssd_bboxes_select(
            rpredictions, rlocalisations, ssd_anchors,
            select_threshold=select_threshold, img_shape=net_shape, num_classes=21, decode=True)
    
    rbboxes = np_methods.bboxes_clip(rbbox_img, rbboxes)
    rclasses, rscores, rbboxes = np_methods.bboxes_sort(rclasses, rscores, rbboxes, top_k=400)
    rclasses, rscores, rbboxes = np_methods.bboxes_nms(rclasses, rscores, rbboxes, nms_threshold=nms_threshold)
    # Resize bboxes to original image shape. Note: useless for Resize.WARP!
    rbboxes = np_methods.bboxes_resize(rbbox_img, rbboxes)
    return rclasses, rscores, rbboxes

In [18]:
# Test on some demo image and visualize output.
path = this_dir + '/data/images/'
output_path = this_dir + '/data/images_out/ssd_vgg/'
image_names = sorted(os.listdir(path))

############# Performance measurements block ###########
# import time
# from tqdm import tqdm
# perf_image = this_dir + "/data/images/" + "000542.jpg"
# times = list()
# img = mpimg.imread(perf_image)
# for i in tqdm(range(100)):
#     timestart = time.time()
#     isess.run([image_4d, predictions, localisations, bbox_img], feed_dict={img_input: img})
#     times.append(time.time() - timestart)
# print(times)    
# performance = np.array(times)
# print("Mean:", np.mean(performance))
# print("Median:", np.median(performance))
# print("Q25:", np.percentile(performance, 25))
# print("Q75:", np.percentile(performance, 75))
##############################################################

########### Accuracy block ####################
import json
from tqdm import tqdm
ann_file = this_dir + '/data/annotations/instances_val2017.json'
test_images_path = this_dir + '/data/test_data/'
test_image_names = sorted(os.listdir(path))
cat_id_to_real_id = \
    {1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 13: 12, 14: 13, 15: 14, 16: 15, 17: 16,
     18: 17, 19: 18, 20: 19, 21: 20, 22: 21, 23: 22, 24: 23, 25: 24, 27: 25, 28: 26, 31: 27, 32: 28, 33: 29, 34: 30,
     35: 31, 36: 32, 37: 33, 38: 34, 39: 35, 40: 36, 41: 37, 42: 38, 43: 39, 44: 40, 46: 41, 47: 42, 48: 43, 49: 44,
     50: 45, 51: 46, 52: 47, 53: 48, 54: 49, 55: 50, 56: 51, 57: 52, 58: 53, 59: 54, 60: 55, 61: 56, 62: 57, 63: 58,
     64: 59, 65: 60, 67: 61, 70: 62, 72: 63, 73: 64, 74: 65, 75: 66, 76: 67, 77: 68, 78: 69, 79: 70, 80: 71, 81: 72,
     82: 73, 84: 74, 85: 75, 86: 76, 87: 77, 88: 78, 89: 79, 90: 80}
    
def parse_annotation(ann_file, img_dir, labels=[]):
    all_imgs = dict()
    seen_labels = set()
    with open(ann_file) as f:
        inst = json.load(f)
    boxes = {image['id']:[] for image in inst['images']}
    cats = inst["categories"]
    for ann in inst['annotations']:
        obj = {}
        real_id = cat_id_to_real_id[ann['category_id']]
        obj['name'] = cats[real_id-1]["name"]
        seen_labels.add(obj['name'])
        obj['xmin'] = int(round(ann['bbox'][0]))
        obj['ymin'] = int(round(ann['bbox'][1]))
        obj['xmax'] = int(round(ann['bbox'][0] + ann['bbox'][2]))
        obj['ymax'] = int(round(ann['bbox'][1] + ann['bbox'][3]))
        boxes[ann['image_id']].append(obj)
    for image in inst['images']:
        all_imgs[os.path.join(img_dir,image['file_name'])] =  boxes[image['id']]
    return all_imgs, seen_labels

imgs, labels = parse_annotation(ann_file, test_images_path)

aaaa = test_images_path + test_image_names[0]
img = mpimg.imread(perf_image)
rclasses, rscores, rbboxes = process_image(img)

print(rclasses, rscores, rbboxes)

###############################################



# for image_name in image_names:
#     print(image_name)
#     img = mpimg.imread(path + image_name)
#     rclasses, rscores, rbboxes =  process_image(img)
#     visualization.plt_bboxes(img, rclasses, rscores, rbboxes, output_path+image_name)

[8] [ 0.99764711] [[ 0.00254753  0.          0.9482094   0.9998045 ]]


Mean: 2.01650357723
Median: 2.16040587425
Q25: 1.18544638157
Q75: 2.57627117634
